### A2.2.3. Undefined Behavior in LLVM

> *Undefined behavior permits the optimizer to assume that certain conditions never occur, enabling transformations that would be invalid if those conditions were possible.*

**Explanation:**

LLVM IR has a layered model of undefined behavior, more nuanced than C's single concept:

1. **Immediate Undefined Behavior** — the program has no semantics (e.g., dividing by zero, null dereference). The optimizer may assume this never happens.

2. **`poison` values** — a deferred form of UB. A `poison` value is produced by operations with undefined results (e.g., `add nsw` that overflows). Using `poison` in certain ways (e.g., as a branch condition, store it to memory) triggers immediate UB. Otherwise, `poison` propagates through computations.

3. **`undef` values** — represent an arbitrary bit pattern. Each use of `undef` may independently take any value of its type. (Being phased out in favor of `poison`.)

**Why it matters for optimization:**

Flags like `nsw` (no signed wrap) and `nuw` (no unsigned wrap) on arithmetic instructions tell the optimizer the operation does not overflow. If it does, the result is `poison`, enabling the optimizer to assume wrapping does not occur and apply transformations like loop induction variable simplification.

**Example:**

```llvm
; nsw = no signed wrap: overflow produces poison
%sum = add nsw i32 %a, %b

; The optimizer can assume %a + %b does not overflow,
; enabling algebraic simplifications.
```

In [ ]:
ub_hierarchy = [
    ("Immediate UB", "Program has no semantics", "division by zero, null deref"),
    ("poison", "Deferred UB — propagates until observed", "add nsw overflow, shl beyond bitwidth"),
    ("undef", "Arbitrary bit pattern per use", "uninitialized alloca read (legacy)"),
]

print("LLVM Undefined Behavior Hierarchy:")
print(f"{'Level':<16} {'Meaning':<46} {'Example'}")
print("-" * 90)
for level, meaning, example in ub_hierarchy:
    print(f"{level:<16} {meaning:<46} {example}")

nsw_example = """
; With nsw flag:
%sum = add nsw i32 2147483647, 1   ; result is poison (signed overflow)

; Without nsw flag:
%sum = add i32 2147483647, 1       ; result is -2147483648 (wraps)
"""
print("\nnsw flag example:")
print(nsw_example)

**References:**

[📘 Lee, J. et al. (2017). *Taming Undefined Behavior in LLVM.* PLDI '17.](https://dl.acm.org/doi/10.1145/3062341.3062343)

[📘 LLVM Project. *LLVM Language Reference Manual — Poison Values.*](https://llvm.org/docs/LangRef.html#poison-values)

---

[⬅️ Previous: LLVM IR Instructions](./02_llvm_intermediate_representation_instructions.ipynb) | [Next: New Pass Manager ➡️](../03_Pass_Infrastructure/01_new_pass_manager.ipynb)